In [11]:
from ozclim import tools
import xarray as xr 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy import stats 
import re 

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Open all wind-speed data
ref = tools.get_ensemble_ds("ref", "MetUV") # Present-day
refshort = tools.get_ensemble_ds("refshort", "MetUV") # High-warming
polshort = tools.get_ensemble_ds("pol3.7short", "MetUV") # Low-warming

['/Users/emmie/Documents/OZCLIM_local/data/GCHP_CAM_c48_20240312/GCHP.w10_ref_c48.MetUV.nc4']
['/Users/emmie/Documents/OZCLIM_local/data/GCHP_CAM_c48_20240312/GCHP.w13_ref_c48.MetUV.nc4']
['/Users/emmie/Documents/OZCLIM_local/data/GCHP_CAM_c48_20240312/GCHP.w14_ref_c48.MetUV.nc4']
['/Users/emmie/Documents/OZCLIM_local/data/GCHP_CAM_c48_20240312/GCHP.w26_ref_c48.MetUV.nc4']
['/Users/emmie/Documents/OZCLIM_local/data/GCHP_CAM_c48_20240312/GCHP.w28_ref_c48.MetUV.nc4']
['/Users/emmie/Documents/OZCLIM_local/data/GCHP_CAM_c48_20240312/GCHP.w10_refshort_c48.MetUV.nc4']
['/Users/emmie/Documents/OZCLIM_local/data/GCHP_CAM_c48_20240312/GCHP.w13_refshort_c48.MetUV.nc4']
['/Users/emmie/Documents/OZCLIM_local/data/GCHP_CAM_c48_20240312/GCHP.w14_refshort_c48.MetUV.nc4']
['/Users/emmie/Documents/OZCLIM_local/data/GCHP_CAM_c48_20240312/GCHP.w26_refshort_c48.MetUV.nc4']
['/Users/emmie/Documents/OZCLIM_local/data/GCHP_CAM_c48_20240312/GCHP.w28_refshort_c48.MetUV.nc4']
['/Users/emmie/Documents/OZCLIM_loc

In [5]:
# Extract U and V components of surface wind
ref_U = ref['Met_U']
ref_V = ref['Met_V']
refshort_U = refshort['Met_U']
refshort_V = refshort['Met_V']
polshort_U = polshort['Met_U']
polshort_V = polshort['Met_V']


In [6]:
# Calculate windspeed
ref_WS = np.sqrt(np.square(ref_U)+np.square(ref_V))
refshort_WS = np.sqrt(np.square(refshort_U)+np.square(refshort_V))
polshort_WS = np.sqrt(np.square(polshort_U)+np.square(polshort_V))

In [7]:
present_diff = ref_WS
future_diff = refshort_WS
futurepol_diff = polshort_WS

In [8]:
present_diff_noland = tools.mask_ocean_c48(present_diff)
future_diff_noland = tools.mask_ocean_c48(future_diff)
futurepol_diff_noland = tools.mask_ocean_c48(futurepol_diff)

In [9]:
present_diff_noland = present_diff_noland.resample(time='1M').mean(dim='time')
future_diff_noland = future_diff_noland.resample(time='1M').mean(dim='time')
futurepol_diff_noland = futurepol_diff_noland.resample(time='1M').mean(dim='time')

In [10]:
tools.set_matplotlib_font("Arial")
fig, ax = plt.subplots(
    figsize=(7, 4.8), 
    dpi=1200, 
    sharex=False, sharey=False,
    nrows=2, ncols=3,
    constrained_layout=True,
)
ax = ax.flatten()
titles= ["a. Eastern North America (Jul)", "b. West-Central Europe (Jul)", "c. Eastern Asia (Jul)",
         "d. Eastern North America (Dec)", "e. West-Central Europe (Dec)", "f. Eastern Asia (Dec)"]
months = [7,7,7, 12,12,12]
ar6_region = {
        "ENA": 5,
        "EAS": 35,
        "WCE": 17,
    }

for _, (region, indices) in enumerate(zip(["ENA", "WCE", "EAS"], [[0,3],[1,4],[2,5]])):

    present_diff_noland_region = tools.crop_regionmask_ar6_c48(present_diff_noland, region_num=ar6_region[region])
    future_diff_noland_region = tools.crop_regionmask_ar6_c48(future_diff_noland, region_num=ar6_region[region])  
    futurepol_diff_noland_region = tools.crop_regionmask_ar6_c48(futurepol_diff_noland, region_num=ar6_region[region])    

    for idx, month in enumerate([7,12]):
        present_diff_noland_region_month = present_diff_noland_region.sel(time=present_diff_noland_region.time.dt.month.isin(month)).isel(lev=0, region=0, drop=True).mean(dim=['nf', 'Ydim', 'Xdim'])
        future_diff_noland_region_month = future_diff_noland_region.sel(time=future_diff_noland_region.time.dt.month.isin(month)).isel(lev=0, region=0, drop=True).mean(dim=['nf', 'Ydim', 'Xdim'])
        futurepol_diff_noland_region_month = futurepol_diff_noland_region.sel(time=futurepol_diff_noland_region.time.dt.month.isin(month)).isel(lev=0, region=0, drop=True).mean(dim=['nf', 'Ydim', 'Xdim'])

        p = present_diff_noland_region_month.values.flatten()
        f = future_diff_noland_region_month.values.flatten()
        fp = futurepol_diff_noland_region_month.values.flatten()

        sns.kdeplot(data=p, ax=ax[indices[idx]], color='black', linestyle='-', fill=False, legend='brief')
        sns.kdeplot(data=f, ax=ax[indices[idx]], color='red', linestyle='--', fill=False, legend='brief')
        sns.kdeplot(data=fp, ax=ax[indices[idx]], color='dodgerblue', linestyle='-.', fill=False, legend='brief')

        _, f_pval_ks = stats.ks_2samp(p, f)
        _, fp_pval_ks = stats.ks_2samp(p, fp)

        formatted_f_pval_ks = format(f_pval_ks, ".3f")
        formatted_fp_pval_ks = format(fp_pval_ks, ".3f")

        ax[indices[idx]].text(.02, 1.11, "p (KS-test) ="+formatted_f_pval_ks, ha='left', va='top', transform=ax[indices[idx]].transAxes, color='red', fontsize=8)
        ax[indices[idx]].text(.02, 1.03, "p (KS-test) ="+formatted_fp_pval_ks, ha='left', va='top', transform=ax[indices[idx]].transAxes, color='dodgerblue', fontsize=8)

        ax[indices[idx]].text(.02, 1.21, titles[indices[idx]], ha='left', va='top', transform=ax[indices[idx]].transAxes, color='k', fontsize=10)

        # Setting plot attributes
        ax[indices[idx]].spines[['right', 'top']].set_visible(False)
        ax[indices[idx]].set_xlabel(None)
        ax[indices[idx]].set_ylabel(None)

xlabel = "Wind Speed (m/s)"
ylabel = "Probability Density"

fig.supylabel(ylabel, fontsize=12)
fig.supxlabel(xlabel, fontsize=12)

plt.savefig("/Users/emmie/Documents/OZCLIM_local/figures/FigureS13.png", format="png")